### Green Life Index

The **Green Life Index** measures how strongly each Warsaw district is connected to nature and outdoor spaces.
It’s based on anonymized network data combined with geographic information about parks, forests, and green areas.
Districts where more user activity occurs within or near green zones receive higher scores, reflecting greater access to and engagement with natural environments.
This index highlights how residents interact with the city’s green infrastructure as part of their daily lives.

In [12]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

In [13]:
df = pd.read_csv("./data/hackplay_warszawa_with_districts.csv")


In [15]:
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["cell_lon"], df["cell_lat"]),
    crs="EPSG:4326",
)

In [16]:
import osmnx as ox

green = ox.features_from_place(
    "Warszawa, Polska",
    tags={
        "leisure": ["park", "garden", "recreation_ground"],
        "landuse": ["forest", "grass", "meadow"]
    }
)
green = green[green.geometry.type.isin(["Polygon", "MultiPolygon"])].to_crs("EPSG:4326")

green = green.explode(index_parts=False, ignore_index=True)
green = green.set_geometry(green.geometry.buffer(0))
green = green.reset_index(drop=True)
green["green_id"] = green.index

In [17]:
joined = gpd.sjoin(
    gdf,
    green[["green_id", "geometry"]],
    how="left",
    predicate="intersects",
)

joined["is_green"] = joined["green_id"].notna()

In [19]:
green_stats = (
    joined.groupby("district", as_index=False)
          .agg(
              total_obs=("user_id", "count"),
              green_obs=("is_green", "sum"),
              unique_users=("user_id", "nunique")
          )
)
green_stats["green_ratio"] = (green_stats["green_obs"] / green_stats["total_obs"]).fillna(0)
green_stats["green_life_score"] = (green_stats["green_ratio"] * 100).round(1)

green_stats = green_stats.sort_values("green_life_score", ascending=False).reset_index(drop=True)
green_stats.to_csv("./data/warsaw_green_places", index=False)


         district  total_obs  green_obs  unique_users  green_ratio  \
0      białołęka        256         52            30     0.203125   
1           ursus         65         10             8     0.153846   
2        mokotów        269         40            31     0.148699   
3       targówek        114          4            17     0.035088   
4       żoliborz         67          1            11     0.014925   
5            wola        374          3            34     0.008021   
6        ursynów        276          2            21     0.007246   
7  śródmieście        361          1            53     0.002770   
8  praga południe        285          0            32     0.000000   
9   praga północ        119          0            15     0.000000   

   green_life_score  
0              20.3  
1              15.4  
2              14.9  
3               3.5  
4               1.5  
5               0.8  
6               0.7  
7               0.3  
8               0.0  
9        